超时强制退出程序

Wrap a function and cancel it it after given timeout.
1. signal
2. multiprocess

## [signal](https://docs.python.org/3/library/signal.html)

refer to: http://www.cnblogs.com/vamei/archive/2012/10/06/2712683.html

信号是由内核(kernel)管理的。信号的产生方式多种多样，它可以是内核自身产生的，比如出现硬件错误(比如出现分母为0的除法运算，或者出现segmentation fault)，内核需要通知某一进程；也可以是其它进程产生的，发送给内核，再由内核传递给目标进程。内核中针对每一个进程都有一个表存储相关信息(房间的信箱)。当内核需要将信号传递给某个进程时，就在该进程相对应的表中的适当位置写入信号(塞入纸条)，这样，就生成(generate)了信号。当该进程执行系统调用时，在系统调用完成后退出内核时，都会顺便查看信箱里的信息。如果有信号，进程会执行对应该信号的操作(signal action, 也叫做信号处理signal disposition)，此时叫做执行(deliver)信号。从信号的生成到信号的传递的时间，信号处于等待(pending)状态(纸条还没有被查看)。我们同样可以设计程序，让其生成的进程阻塞(block)某些信号，也就是让这些信号始终处于等待的状态，直到进程取消阻塞(unblock)或者无视信号。

- *SIGINT*   当键盘按下CTRL+C从shell中发出信号，信号被传递给shell中前台运行的进程，对应该信号的默认操作是中断 (INTERRUPT) 该进程。
- *SIGQUIT*  当键盘按下CTRL+\从shell中发出信号，信号被传递给shell中前台运行的进程，对应该信号的默认操作是退出 (QUIT) 该进程。
- *SIGTSTP*  当键盘按下CTRL+Z从shell中发出信号，信号被传递给shell中前台运行的进程，对应该信号的默认操作是暂停 (STOP) 该进程。
- *SIGCONT*  用于通知暂停的进程继续。
- *SIGALRM*  起到定时器的作用，通常是程序在一定的时间之后才生成该信号。




编号
名称
作用

1
SIGHUP
终端挂起或者终止进程。默认动作为终止进程

2
SIGINT
键盘中断 <ctrl+c> 经常会用到。默认动作为终止进程

3
SIGQUIT
键盘退出键被按下。一般用来响应 <ctrl+d>。 默认动作终止进程

9
SIGKILL
强制退出。 shell中经常使用

14
SIGALRM
定时器超时，默认为终止进程

15
SIGTERM
程序结束信号，程序一般会清理完状态在退出，我们一般说的优雅的退出

? 
SIGSTP
<ctrl+z>,默认操作是暂停 (STOP) 该进程

### SIGALRM
signal.alarm()，用于在一定时间之后，向进程自身发送SIGALRM信号

In [2]:
import signal

def loop_forever():
    import time
    while 1:
        print('Tick')
        time.sleep(1)
        
def handler(signum, frame):
    print("Forever is over!")
    raise Exception("end of time")

In [3]:
# 十秒钟后，调用 handler

# register signal.SIGALRM's handler 
signal.signal(signal.SIGALRM, handler)

signal.alarm(10)

0

In [4]:
try:
    loop_forever()
except Exception as e:
    print(e)

Tick
Tick
Tick
Tick
Tick
Tick
Tick
Tick
Tick
Tick
Forever is over!
end of time


In [5]:
# Cancel the timer if the function returned before timeout
signal.alarm(0)

0

## thread

refrer to: https://stackoverflow.com/questions/492519/timeout-on-a-function-call

1. threading.Timer

Call a function after a specified number of seconds:

Usage: 

```
threading.Timer(interval, function, args=None, kwargs=None)

t = Timer(30.0, f, args=None, kwargs=None)
t.start()
t.cancel()     # stop the timer's action if it's still waiting
```
2. use thread.interrupt_main() to raise KeyboardInterupt and quit function

### `exit_after` decorator

In [15]:
import threading
from threading import Thread
import sys
import _thread as thread
from time import sleep

def quit_function(fn_name):
    print('function {} exceed time limit'.format(fn_name), file=sys.stderr)
    sys.stderr.flush()
    thread.interrupt_main() # raise KeyboardInterupt

def exit_after(timeout):
    '''
    use as decorator to exit process if function takes longer than timeout(seconds)
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(timeout, quit_function, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer
    
# countdown(8)

In [16]:
# Usage
@exit_after(5)
def countdown(n):
    print('start ...', flush=True)
    for i in range(n, -1, -1):
        print(i, end=', ', flush=True)
        sleep(1)
        
    print('\nfinished ...')

In [17]:
countdown(3)
countdown(10)

start ...
3, 2, 1, 0, 
finished ...
start ...
10, 9, 8, 7, 6, 

function countdown exceed time limit


KeyboardInterrupt: 

## multiprocessing

Use `multiprocessing.Process`

Process objects represent activity that is run in a separate process.
The class is analogous to `threading.Thread`.


`p.join(timeout=None)`: Wait until child process terminates


In [18]:
import multiprocessing


def bar():
    for i in range(100):
        print('Tick')
        sleep(1)

def exit_after(timeout=10):
    # start bar as a process
    p = multiprocessing.Process(target=bar)
    p.start()
    
    # wait for timeout seconds or until process finishes
    p.join(timeout)
    
    # if thred is still active
    if p.is_alive():
        print('process is still running... kill it ...')
        # terminate
        p.terminate()
        p.join()
    

In [19]:
exit_after()

Tick
Tick
Tick
Tick
Tick
Tick
Tick
Tick
Tick
Tick
process is still running... kill it ...
